# SARSA (Lambda) [In-Progress]

In [12]:
import gym 
import numpy as np 
import time

In [19]:
env = gym.make("Taxi-v3", render_mode = "human")
env.reset()
env.render()

In [20]:
state_space = env.observation_space
action_space = env.action_space

print("We have {} action space and {} state space".format(action_space, state_space))

We have Discrete(6) action space and Discrete(500) state space


In [10]:
#This function returns an array of action probabilities for a given state (a polic) 
#this policy is designed to be epsilon-greedy in relation to the state action value function Q 
def policy_fn(Q, num_actions, e, state):
    action_probabilities = np.ones(num_actions) * (e/num_actions)
    highest_action_value = np.argmax(Q[state])
    action_probabilities[highest_action_value] += (1 - e)
    
    return action_probabilities

def update_params(Q, state, action, reward, s_next, alpha, gamma, e) -> None: 
    old_val = Q[state, action]
    pi = policy_fn(Q, action_space.n, e, state)
    a_next = np.random.choice(np.arange(action_space.n), p = pi)
    new_val = old_val + alpha * (reward + gamma * a_next - old_val)
    Q[state, action] = new_val

In [5]:
#Initializations 
Q = np.zeros((state_space.n, action_space.n))
returns = [[[]]*action_space.n for i in range(state_space.n)]
pi = np.zeros((state_space.n, action_space.n))
epochs_per_episode = []
action_prev = 0 
state_prev = 0 

#hyperparams 
num_episodes = 600
epsilon = 0.1
gamma = 1
alpha = 0.1
lamda = 0.6 

In [ ]:
for i in range(num_episodes):
    E = np.zeros((state_space.n, action_space.n))
    episode = []
    state = env.reset()
    action = 0 
    cumulative_reward = 0 
    epoch = 0
    terminated = False 
    print("Episode: {} Epoch: {}".format(i, epoch))
    
    while not terminated: 

        epoch+=1
        if type(state)==tuple: 
            state = state[0]
        
        next_state, reward, terminated, truncated, step_dict = env.step(action)
        
        pi[state] = policy_fn(Q, action_space.n, epsilon, state)
        next_action = np.random.choice(np.arange(action_space.n), p = pi[state])
    
         
        episode.append((state, action, reward))
        
        if epoch == 1: 
            action_prev = action
            continue 
        
        
        update_params(Q, state, action, reward, next_state, alpha, gamma, epsilon)
        
        if terminated: 
            print("Episode: {} Epoch: {}".format(i, epoch))
            epochs_per_episode.append(epoch)
        
        state = next_state
        action = next_action